# Convolutional Neural Nets

Let's look at constructing a convolutional neural net. We can use the same MNIST number dataset and classify the data using CNNs

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from matplotlib import cm

#Want to use version of Tensorflow > 2.0
print('Using Tensorflow version %s' % tf.__version__)

Download MNIST dataset

In [ ]:
#Load the dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Intensity of pixels ranges from 0-255. We need to normalise them
x_train, x_test = x_train / 255.0, x_test / 255.0


In [ ]:
# Reshaping the array to 4-dims so that it can work with the tf.keras.layers.Conv2D API
#The 4 dims are , Num_of Events,image_height, image_width, num_of_channels
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

#Split into training and validation samples
x_train, x_val = x_train[0:55000], x_train[55000:]
y_train, y_val = y_train[0:55000], y_train[55000:]

In [ ]:
# Creating a Sequential Model and adding the layers

model = tf.keras.models.Sequential([
    #tf.keras.layers.Conv2D(number_of_filters, kernel_size=(filter size), input_shape=(height, width, num_of_channels))
    tf.keras.layers.Conv2D(28, kernel_size=(3,3), input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(), # Flattening the 2D arrays for fully connected layers
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])

model.summary()

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

num_epochs = 10
batchSize = 1000

model.fit(x_train, y_train, 
          validation_data=(x_val, y_val),
          batch_size=batchSize,
          epochs=num_epochs)
